In [6]:
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict


date = "241030"
box1 = "16"
box2 = "17"
exp  = "WT_PTZ"
name = date+"_"+box1+"_"+box2+"_"+exp
path = name+"/"



# Load files
video_path = path+name+'_Box1_0001.avi'


# Parameters
# video_path = '/mnt/data/241031_16_17_WT_ALDH7A1_PTZ_Box2_0001.avi'
num_wells = 96
rows, cols = 8, 12  # Assuming an 8x12 grid for 96 wells

# Load video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Calculate dimensions for each well
well_width = frame_width // cols
well_height = frame_height // rows

# Initialize background subtractor to detect moving fish
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Dictionary to hold traces for each well
traces = defaultdict(list)  # Stores coordinates for each well across frames
frame_data_list = []

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction to isolate moving objects (fish)
    fg_mask = bg_subtractor.apply(frame)
    _, thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
    
    # Data dictionary to store coordinates of fish per well for this frame
    frame_data = {}
    
    # Loop over each well in the grid
    for r in range(rows):
        for c in range(cols):
            well_id = r * cols + c + 1
            
            # Define boundaries for the well
            x_start, y_start = c * well_width, r * well_height
            x_end, y_end = x_start + well_width, y_start + well_height
            well_region = thresh[y_start:y_end, x_start:x_end]
            
            # Find contours in this well region
            contours, _ = cv2.findContours(well_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Store fish coordinates in this well
            fish_coords = []
            for contour in contours:
                if cv2.contourArea(contour) > 10:  # Adjust to ignore small noise
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"]) + x_start
                        cy = int(M["m01"] / M["m00"]) + y_start
                        fish_coords.append((cx, cy))
                        traces[well_id].append((cx, cy))
            
            # Record coordinates for this frame
            frame_data[well_id] = fish_coords
    
    frame_data_list.append(frame_data)
    frame_idx += 1

    # Optional: Break early for testing (e.g., after first 500 frames)
    # if frame_idx > 500:
    #     break

cap.release()

# Save traces to a DataFrame for each well
trace_data = {
    well_id: pd.DataFrame(coords, columns=['x', 'y'])
    for well_id, coords in traces.items()
}

# Save coordinates per frame to a DataFrame
coordinates_df = pd.DataFrame(frame_data_list)

# Save trace data and per-frame coordinates to files (optional)
# for well_id, df in trace_data.items():
#     df.to_csv(f'well_{well_id}_trace.csv', index=False)

# coordinates_df.to_csv('frame_coordinates.csv', index=False)

print("Processing complete. Trace files and frame coordinates saved.")

Processing complete. Trace files and frame coordinates saved.


In [9]:
trace_data


{26:        x    y
 0    195  311
 1    197  312
 2    198  312
 3    127  303
 4    127  303
 ..   ...  ...
 583  159  346
 584  159  346
 585  159  346
 586  159  346
 587  159  346
 
 [588 rows x 2 columns],
 51:        x    y
 0    260  542
 1    259  542
 2    245  538
 3    259  542
 4    259  542
 ..   ...  ...
 569  298  558
 570  298  558
 571  298  558
 572  298  558
 573  298  558
 
 [574 rows x 2 columns],
 52:        x    y
 0    399  549
 1    398  549
 2    398  549
 3    322  589
 4    322  587
 ..   ...  ...
 992  382  532
 993  382  531
 994  382  531
 995  382  531
 996  382  531
 
 [997 rows x 2 columns],
 79:         x    y
 0     723  813
 1     684  763
 2     670  791
 3     684  771
 4     670  781
 ...   ...  ...
 996   670  792
 997   735  814
 998   714  728
 999   719  832
 1000  669  787
 
 [1001 rows x 2 columns],
 16:        x    y
 0    342  202
 1    343  202
 2    336  201
 3    335  201
 4    336  200
 ..   ...  ...
 314  335  238
 315  335  238
 316

In [15]:
import cv2
import numpy as np

# Parameters
# video_path = '/mnt/data/241031_16_17_WT_ALDH7A1_PTZ_Box2_0001.avi'
rows, cols = 8, 12
occupied_wells = defaultdict(int)  # Count activity frames for each well

# Load video and calculate well dimensions
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
well_width = frame_width // cols
well_height = frame_height // rows

# Background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Settings for analysis
num_frames_to_check = 500  # Increase frames analyzed
activity_threshold = 0.8  # Threshold for determining if a well is occupied (80% of frames)

# Process frames
for frame_idx in range(num_frames_to_check):
    ret, frame = cap.read()
    if not ret:
        break

    # Background subtraction and thresholding
    fg_mask = bg_subtractor.apply(frame)
    _, thresh = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Analyze each well
    for r in range(rows):
        for c in range(cols):
            well_id = f"{chr(65 + r)}{c + 1}"
            x_start, y_start = c * well_width, r * well_height
            x_end, y_end = x_start + well_width, y_start + well_height
            well_region = thresh[y_start:y_end, x_start:x_end]

            # Find contours in the well region
            contours, _ = cv2.findContours(well_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                if cv2.contourArea(contour) > 10:  # Filter small areas as noise
                    occupied_wells[well_id] += 1  # Increment frame count for activity
                    break  # No need to check further if movement is detected in this well

cap.release()

# Determine consistently occupied wells based on activity threshold
occupied_wells = [
    well for well, count in occupied_wells.items()
    if count / num_frames_to_check >= activity_threshold
]

print("Consistently occupied wells:", sorted(occupied_wells))

Consistently occupied wells: []


In [13]:
occupied_wells

[7, 16, 26, 51, 52, 79, 93, 94, 96]